In [2]:
import pandas as pd
import numpy as np  # Ci servirà più avanti, ma importalo ora

# Carica il tuo dataset
# Assicurati di usare il percorso e il nome corretti del tuo file
try:
    df = pd.read_csv('data/fc25/all_players.csv')
    print(f"Dataset caricato con successo: {df.shape[0]} righe e {df.shape[1]} colonne.")
except FileNotFoundError:
    print("Errore: File non trovato. Controlla il percorso e il nome del file.")

Dataset caricato con successo: 17737 righe e 58 colonne.


In [3]:
print("Nomi delle colonne:", df.columns.to_list())

Nomi delle colonne: ['Unnamed: 0.1', 'Unnamed: 0', 'Rank', 'Name', 'OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'Acceleration', 'Sprint Speed', 'Positioning', 'Finishing', 'Shot Power', 'Long Shots', 'Volleys', 'Penalties', 'Vision', 'Crossing', 'Free Kick Accuracy', 'Short Passing', 'Long Passing', 'Curve', 'Dribbling', 'Agility', 'Balance', 'Reactions', 'Ball Control', 'Composure', 'Interceptions', 'Heading Accuracy', 'Def Awareness', 'Standing Tackle', 'Sliding Tackle', 'Jumping', 'Stamina', 'Strength', 'Aggression', 'Position', 'Weak foot', 'Skill moves', 'Preferred foot', 'Height', 'Weight', 'Alternative positions', 'Age', 'Nation', 'League', 'Team', 'play style', 'url', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes']


### DATAFRAME SEPARATI 

In [4]:
# Definisci le tue variabili (facilita la lettura)
colonna_posizione = 'Position'
sigla_portiere = 'GK'

# 1. Creare il DataFrame dei Portieri
# Seleziona tutte le righe dove il valore di 'colonna_posizione' è UGUALE a 'sigla_portiere'
df_portieri = df[df[colonna_posizione] == sigla_portiere].copy()

# 2. Creare il DataFrame dei Giocatori di Movimento
# Seleziona tutte le righe dove il valore di 'colonna_posizione' è DIVERSO da 'sigla_portiere'
df_movimento = df[df[colonna_posizione] != sigla_portiere].copy()

In [5]:
print(f"\n--- Verifica Separazione ---")

# Controlla il DataFrame dei portieri
print(f"Numero totale di portieri: {len(df_portieri)}")
# Stampa le posizioni uniche per essere sicuro
print(f"Posizioni trovate nel df_portieri: {df_portieri[colonna_posizione].unique()}")

# Controlla il DataFrame dei giocatori di movimento
print(f"Numero totale di giocatori di movimento: {len(df_movimento)}")
# Stampa le posizioni uniche per essere sicuro
print(f"Posizioni trovate nel df_movimento: {df_movimento[colonna_posizione].unique()}")

# Controllo finale: la somma deve fare il totale
print(f"Controllo: {len(df_portieri)} + {len(df_movimento)} = {len(df_portieri) + len(df_movimento)}")
print(f"Righe totali originali: {len(df)}")


--- Verifica Separazione ---
Numero totale di portieri: 1999
Posizioni trovate nel df_portieri: ['GK']
Numero totale di giocatori di movimento: 15738
Posizioni trovate nel df_movimento: ['ST' 'CDM' 'CAM' 'LW' 'CM' 'CB' 'RW' 'LB' 'LM' 'RB' 'RM']
Controllo: 1999 + 15738 = 17737
Righe totali originali: 17737


### PULIZIA COLONNE

### Giocatori Movimento

In [6]:
# Colonne specifiche dei portieri (piene di NaN)
colonne_stats_gk = ['GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes']

# Colonne "spazzatura" identificate da te
colonne_inutili = ['Unnamed: 0.1', 'Unnamed: 0', 'Rank', 'url']

# Combiniamo le liste
colonne_da_droppare_movimento = colonne_stats_gk + colonne_inutili

# Applichiamo il .drop()
df_movimento_pulito = df_movimento.drop(columns=colonne_da_droppare_movimento)

### Portieri

In [7]:
# --- CREAZIONE df_portieri_pulito (Versione Finale) ---

# 1. (Richiesta 1) Definiamo le colonne "generali" da TENERE
#    NOTA: 'Alternative positions' è stata esclusa da questa lista.
colonne_generali = [
    'Name', 'OVR', 'Position', 'Age', 'Nation', 'League', 'Team', 
    'Height', 'Weight', 'Preferred foot', 'Weak foot', 'Skill moves',
    'play style'
]

# 2. Definiamo le statistiche VERE da tenere
colonne_stats_portiere_vere = [
    'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes'
]
colonna_velocita_gk = ['DEF'] # La teniamo temporaneamente con questo nome

# 3. Creiamo un DataFrame temporaneo con solo le colonne che ci servono
colonne_da_tenere = colonne_generali + colonne_stats_portiere_vere + colonna_velocita_gk
df_portieri_temp = df_portieri[colonne_da_tenere].copy()

# 4. (Richiesta 2) Rinominiamo 'DEF' in 'GK Speed'
df_portieri_temp = df_portieri_temp.rename(columns={'DEF': 'GK Speed'})

# 5. (Richiesta 3) Definiamo l'ORDINE FINALE delle colonne
#    Mettiamo le 6 statistiche chiave subito dopo 'Position'
colonne_ordine_finale = [
    'Name', 'OVR',  
    'GK Diving', 'GK Handling', 'GK Kicking', 'GK Reflexes', 'GK Speed', 'GK Positioning', 
    'Position', 'Age', 'Nation', 'League', 'Team', 'Height', 'Weight', 
    'Preferred foot', 'Weak foot', 'Skill moves', 'play style'
]


# 6. Creiamo il DataFrame finale pulito applicando il nuovo ordine
df_portieri_pulito = df_portieri_temp[colonne_ordine_finale]

# --- VERIFICA FINALE ---
print("--- df_portieri_pulito (Verificato e Riorganizzato) ---")
print(f"Numero di colonne: {len(df_portieri_pulito.columns)}")
df_portieri_pulito.head()

--- df_portieri_pulito (Verificato e Riorganizzato) ---
Numero di colonne: 19


,Name,OVR,GK Diving,GK Handling,GK Kicking,GK Reflexes,GK Speed,GK Positioning,Position,Age,Nation,League,Team,Height,Weight,Preferred foot,Weak foot,Skill moves,play style
8,Gianluigi Donnarumma,89,90.0,84.0,73.0,90.0,52,87.0,GK,25,Italy,Ligue 1 McDonald's,Paris SG,"196cm / 6'5""",90kg / 198lb,Right,3,1,"Deflector+, Footwork"
9,Alisson,89,86.0,85.0,85.0,89.0,56,90.0,GK,31,Brazil,Premier League,Liverpool,"193cm / 6'4""",91kg / 201lb,Right,3,1,"Deflector+, 1v1 Close Down, Far Throw"
10,Thibaut Courtois,89,85.0,89.0,76.0,90.0,46,88.0,GK,32,Belgium,LALIGA EA SPORTS,Real Madrid,"200cm / 6'7""",96kg / 212lb,Left,3,1,"1v1 Close Down+, Cross Claimer, Far Throw"
13,Marc-André ter Stegen,89,86.0,85.0,89.0,91.0,47,86.0,GK,32,Germany,LALIGA EA SPORTS,FC Barcelona,"187cm / 6'2""",85kg / 187lb,Right,4,1,"Footwork+, 1v1 Close Down"
21,Ederson,88,87.0,82.0,91.0,87.0,64,86.0,GK,31,Brazil,Premier League,Manchester City,"188cm / 6'2""",86kg / 190lb,Left,3,1,"Long Ball Pass+, Cross Claimer, Pinged Pass"


In [8]:
print("--- 1. VERIFICA PORTIERI (df_portieri_pulito) ---")

# 1A. Controllo Colonne
print("\n[Controllo Colonne (info)]: ")
df_portieri_pulito.info()

# 1B. Controllo Righe (Corretto: 'Position' singolare)
print("\n[Controllo Righe (Position)]: ")
print(df_portieri_pulito['Position'].value_counts())

# 1C. Ispezione visiva
print("\n[Ispezione prime 5 righe]: ")
df_portieri_pulito.head()

--- 1. VERIFICA PORTIERI (df_portieri_pulito) ---

[Controllo Colonne (info)]: 
<class 'pandas.core.frame.DataFrame'>
Index: 1999 entries, 8 to 17722
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Name            1999 non-null   object 
 1   OVR             1999 non-null   int64  
 2   GK Diving       1999 non-null   float64
 3   GK Handling     1999 non-null   float64
 4   GK Kicking      1999 non-null   float64
 5   GK Reflexes     1999 non-null   float64
 6   GK Speed        1999 non-null   int64  
 7   GK Positioning  1999 non-null   float64
 8   Position        1999 non-null   object 
 9   Age             1999 non-null   int64  
 10  Nation          1999 non-null   object 
 11  League          1999 non-null   object 
 12  Team            1999 non-null   object 
 13  Height          1999 non-null   object 
 14  Weight          1999 non-null   object 
 15  Preferred foot  1999 non-null   object 
 16  We

,Name,OVR,GK Diving,GK Handling,GK Kicking,GK Reflexes,GK Speed,GK Positioning,Position,Age,Nation,League,Team,Height,Weight,Preferred foot,Weak foot,Skill moves,play style
8,Gianluigi Donnarumma,89,90.0,84.0,73.0,90.0,52,87.0,GK,25,Italy,Ligue 1 McDonald's,Paris SG,"196cm / 6'5""",90kg / 198lb,Right,3,1,"Deflector+, Footwork"
9,Alisson,89,86.0,85.0,85.0,89.0,56,90.0,GK,31,Brazil,Premier League,Liverpool,"193cm / 6'4""",91kg / 201lb,Right,3,1,"Deflector+, 1v1 Close Down, Far Throw"
10,Thibaut Courtois,89,85.0,89.0,76.0,90.0,46,88.0,GK,32,Belgium,LALIGA EA SPORTS,Real Madrid,"200cm / 6'7""",96kg / 212lb,Left,3,1,"1v1 Close Down+, Cross Claimer, Far Throw"
13,Marc-André ter Stegen,89,86.0,85.0,89.0,91.0,47,86.0,GK,32,Germany,LALIGA EA SPORTS,FC Barcelona,"187cm / 6'2""",85kg / 187lb,Right,4,1,"Footwork+, 1v1 Close Down"
21,Ederson,88,87.0,82.0,91.0,87.0,64,86.0,GK,31,Brazil,Premier League,Manchester City,"188cm / 6'2""",86kg / 190lb,Left,3,1,"Long Ball Pass+, Cross Claimer, Pinged Pass"


In [9]:
# --- 6. CONVERSIONE TIPO DI DATO (da float a int) ---
#    (Questo blocco va DOPO il blocco 4 e 5)

# 1. Definiamo le 6 statistiche dei portieri che vogliamo convertire
statistiche_gk_da_convertire = [
    'GK Speed', 'GK Diving', 'GK Handling', 
    'GK Kicking', 'GK Positioning', 'GK Reflexes'
]

# 2. Creiamo un dizionario di conversione
#    Questo dice a Pandas: "voglio che 'GK Speed' sia 'int', 'GK Diving' sia 'int', ecc."
conversion_dict = {col: int for col in statistiche_gk_da_convertire}

# 3. Applichiamo la conversione
#    Usiamo .astype() per applicare il nostro dizionario.
#    Questo è il modo più pulito e sicuro per farlo.
df_portieri_pulito = df_portieri_pulito.astype(conversion_dict)

# --- 7. VERIFICA FINALE TIPO DATI ---
print("\n--- Verifica Tipi di Dato (Portieri) ---")
# Ora nell'output di .info() dovresti vedere 'int64' per le 6 statistiche,
# invece di 'float64'.
df_portieri_pulito.info()


--- Verifica Tipi di Dato (Portieri) ---
<class 'pandas.core.frame.DataFrame'>
Index: 1999 entries, 8 to 17722
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Name            1999 non-null   object
 1   OVR             1999 non-null   int64 
 2   GK Diving       1999 non-null   int64 
 3   GK Handling     1999 non-null   int64 
 4   GK Kicking      1999 non-null   int64 
 5   GK Reflexes     1999 non-null   int64 
 6   GK Speed        1999 non-null   int64 
 7   GK Positioning  1999 non-null   int64 
 8   Position        1999 non-null   object
 9   Age             1999 non-null   int64 
 10  Nation          1999 non-null   object
 11  League          1999 non-null   object
 12  Team            1999 non-null   object
 13  Height          1999 non-null   object
 14  Weight          1999 non-null   object
 15  Preferred foot  1999 non-null   object
 16  Weak foot       1999 non-null   int64 
 17  Skill moves   

In [10]:
df_portieri_pulito.head()

,Name,OVR,GK Diving,GK Handling,GK Kicking,GK Reflexes,GK Speed,GK Positioning,Position,Age,Nation,League,Team,Height,Weight,Preferred foot,Weak foot,Skill moves,play style
8,Gianluigi Donnarumma,89,90,84,73,90,52,87,GK,25,Italy,Ligue 1 McDonald's,Paris SG,"196cm / 6'5""",90kg / 198lb,Right,3,1,"Deflector+, Footwork"
9,Alisson,89,86,85,85,89,56,90,GK,31,Brazil,Premier League,Liverpool,"193cm / 6'4""",91kg / 201lb,Right,3,1,"Deflector+, 1v1 Close Down, Far Throw"
10,Thibaut Courtois,89,85,89,76,90,46,88,GK,32,Belgium,LALIGA EA SPORTS,Real Madrid,"200cm / 6'7""",96kg / 212lb,Left,3,1,"1v1 Close Down+, Cross Claimer, Far Throw"
13,Marc-André ter Stegen,89,86,85,89,91,47,86,GK,32,Germany,LALIGA EA SPORTS,FC Barcelona,"187cm / 6'2""",85kg / 187lb,Right,4,1,"Footwork+, 1v1 Close Down"
21,Ederson,88,87,82,91,87,64,86,GK,31,Brazil,Premier League,Manchester City,"188cm / 6'2""",86kg / 190lb,Left,3,1,"Long Ball Pass+, Cross Claimer, Pinged Pass"


In [11]:

# Salva i giocatori di movimento in un file
df_movimento_pulito.to_csv('movimento_puliti.csv', index=False)

# Salva i portieri in un altro file
df_portieri_pulito.to_csv('portieri_puliti.csv', index=False)

print("DataFrame puliti salvati in 'movimento_puliti.csv' e 'portieri_puliti.csv'")

DataFrame puliti salvati in 'movimento_puliti.csv' e 'portieri_puliti.csv'
